# Create Multi-AI Agent to Research and Write an Article


Learn about Agents, Tasks and organize agents to work sequentially.

In [1]:
# To install crewAI, you need to have Python >=3.10 and <=3.13 installed on your system:
import sys
print(sys.version)

3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]


In [2]:
!pip install -r requirements.txt

  Obtaining dependency information for websockets==12.0 from https://files.pythonhosted.org/packages/95/aa/75fa3b893142d6d98a48cb461169bd268141f2da8bfca97392d6462a02eb/websockets-12.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached websockets-12.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached websockets-12.0-cp311-cp311-macosx_11_0_arm64.whl (121 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 0.8.0 requires websockets<15.0dev,>=13.0, but you have websockets 12.0 which is incompatible.


In [3]:
pip install google-generativeai crewai langchain langchain-core


Note: you may need to restart the kernel to use updated packages.


### Step 01. Intalling dependencies

Installing crewAi library and crewAI tools

In [2]:
pip install pydantic==2.6.0


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install crewai # Or the latest stable version


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
pip check

streamlit 1.35.0 requires toml, which is not installed.
frontend 0.0.3 requires aiofiles, which is not installed.
pyxnat 1.6.2 requires pathlib, which is not installed.
mdit-py-plugins 0.3.0 has requirement markdown-it-py<3.0.0,>=1.0.0, but you have markdown-it-py 3.0.0.
streamlit 1.35.0 has requirement pillow<11,>=7.1.0, but you have pillow 11.0.0.
googletrans 4.0.0rc1 has requirement httpx==0.13.3, but you have httpx 0.27.0.
conda-repo-cli 1.0.137 has requirement urllib3>=2.2.2, but you have urllib3 2.2.1.
s3fs 2024.12.0 has requirement fsspec==2024.12.0.*, but you have fsspec 2024.5.0.
tensorflow-macos 2.14.0 has requirement wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0.
anaconda-cloud-auth 0.1.3 has requirement pydantic<2.0, but you have pydantic 2.6.0.
spyder 6.0.1 has requirement qtconsole<5.7.0,>=5.6.0, but you have qtconsole 5.5.2.
transformers 4.45.2 has requirement huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 0.23.0.
transformers 4.45.2 has requirement tokenize

In [5]:
from crewai import Agent, Task, Crew

### Step 02. Importing Libraries

Installing crewAi library and crewAI tools

For LLM agents, we will be using OpenAI's gpt-3.5-turbo.

### Step 03. Creating Agents

Create instance of the Agent class with the desired properties, `role`, `goal` and `backstory`

- `role`: agent's function within the crew
- `goal`: agent's objective. Guides agent's decision making process
- `backstory`: provides context to agent's role and goal. Helps in interation and collaboration.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

### Agent: Writer

In [20]:
import google.generativeai as genai
import os
from crewai import Agent
from langchain_core.language_models import LLM
from langchain_core.outputs import LLMResult
from typing import List, Optional, Any

# ✅ Configure Gemini API key
os.environ["GEMINI_API_KEY"] = "Enter your Gemini Api"
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# ✅ Proper LangChain-compatible wrapper for Gemini API
class GeminiLangChainWrapper(LLM):
    model_name: str = "gemini-pro"

    def __init__(self, model_name="gemini-pro", **kwargs):
        super().__init__(**kwargs)  # Ensuring compatibility with LangChain's LLM
        object.__setattr__(self, "model_name", model_name)  # ✅ Fix for Pydantic model restrictions
        object.__setattr__(self, "gemini_model", genai.GenerativeModel(model_name))  # ✅ Proper initialization

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs) -> str:
        response = self.gemini_model.generate_content(prompt)
        return response.text if response else "No response"

    def generate(self, prompts: List[str], stop: Optional[List[str]] = None, **kwargs) -> LLMResult:
        """Required method for compatibility with LangChain"""
        responses = [self._call(prompt) for prompt in prompts]
        return LLMResult(generations=[[{"text": response}] for response in responses])

    def bind(self, **kwargs):
        """CrewAI expects a `bind()` method"""
        return self

    @property
    def _llm_type(self) -> str:
        return "gemini"

# ✅ Create an instance of the wrapper
gemini_llm = GeminiLangChainWrapper()

# ✅ CrewAI Agent using Gemini
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article about {topic}.",
    allow_delegation=False,
    verbose=True,
    llm=gemini_llm  # Use the Gemini wrapper instead of OpenAI's ChatOpenAI
)

# ✅ Example of using the Gemini model
response = gemini_llm._call("Write a short blog post on AI and sustainability.")
print(response)


**Artificial Intelligence: A Catalyst for Sustainability**

In a world grappling with environmental challenges, artificial intelligence (AI) emerges as a powerful ally in our quest for sustainability. With its ability to process vast amounts of data, identify patterns, and optimize complex systems, AI can pave the way for a more sustainable future.

**Monitoring and Data Analysis:**

AI-powered sensors and data analytics enable real-time monitoring of resource consumption, emissions, and other environmental indicators. This data provides insights into the root causes of environmental issues, allowing organizations to take informed actions and track their progress towards sustainability goals.

**Smart Energy Management:**

AI can optimize energy usage by predicting demand, controlling smart grids, and integrating renewable energy sources. By automating these processes, AI reduces energy waste, lowers emissions, and promotes sustainable energy practices.

**Precision Agriculture:**

AI 

In [21]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion pieces on {topic}.",
    backstory=(
        "You are a highly skilled content writer working on a new opinion piece about {topic}. "
        "Your writing is based on the work of the Content Planner, who provides an outline "
        "and relevant context. You follow the planner’s direction while also adding "
        "independent insights backed by research. "
        "You ensure clarity, conciseness, and engagement in your writing. "
        "You differentiate between factual statements and personal opinions, "
        "citing sources where necessary."
    ),
    allow_delegation=False,
    verbose=True,
    llm=gemini_llm  # ✅ Using the Gemini wrapper instead of OpenAI's ChatOpenAI
)


### Agent: Editor

In [22]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion pieces on {topic}.",
    backstory=(
        "You are a highly skilled content writer working on a new opinion piece about {topic}. "
        "Your writing is based on the work of the Content Planner, who provides an outline "
        "and relevant context. You follow the planner’s direction while also adding "
        "independent insights backed by research. "
        "You ensure clarity, conciseness, and engagement in your writing. "
        "You differentiate between factual statements and personal opinions, "
        "citing sources where necessary."
    ),
    allow_delegation=False,
    verbose=True,
    llm=gemini_llm  # ✅ Using the Gemini wrapper instead of OpenAI's ChatOpenAI
)


In [16]:
editor = Agent(
    role="Editor",
    goal="Refine and edit blog posts to align with the organization's writing style, ensuring clarity, consistency, and professionalism.",
    backstory=(
        "You are an experienced editor responsible for reviewing content provided by the Content Writer. "
        "Your goal is to ensure the blog post follows journalistic best practices, adheres to the organization's tone, "
        "and maintains a balance in viewpoints when presenting opinions or assertions. "
        "You check for factual accuracy, readability, and structural flow while ensuring the piece avoids unnecessary controversy. "
        "Your editing enhances engagement, corrects grammar and style inconsistencies, and strengthens the article's impact."
    ),
    allow_delegation=False,
    verbose=True,
    llm=gemini_llm  # ✅ Using Gemini API instead of OpenAI's ChatOpenAI
)


### Step 04. Define the Tasks

Tasks are **specific assignments**, completed by agents. 
They **provide all necessary details for execution**, such as a `description`, `expected_output` , `agent` responsible, tools, etc.

- `description`: Tells about what task entails
- `expected_output`: Tells about what task output should look like
- `agent`: Agent responsible for the task

### Task: Plan

### Task: Write

In [23]:
plan = Task(
    description=(
        "1. Research and prioritize the latest trends, key players, and noteworthy news related to {topic}.\n"
        "2. Define the target audience, considering their demographics, interests, and pain points.\n"
        "3. Develop a structured content outline, including:\n"
        "   - A compelling introduction.\n"
        "   - Key discussion points with supporting details.\n"
        "   - A clear and engaging call to action.\n"
        "4. Integrate SEO keywords and provide relevant sources for accuracy and credibility."
    ),
    expected_output=(
        "A well-structured content plan document that includes:\n"
        "- A detailed outline with introduction, key points, and conclusion.\n"
        "- Audience analysis and insights.\n"
        "- SEO keyword recommendations.\n"
        "- Reliable data sources and references."
    ),
    agent=planner  # ✅ Ensuring 'planner' is a valid Agent instance
)


In [13]:
write = Task(
    description=(
        "1. Use the content plan as a foundation to craft an engaging and informative blog post on {topic}.\n"
        "2. Naturally integrate relevant SEO keywords to optimize search visibility.\n"
        "3. Structure the blog post with clear, engaging sections and well-named subtitles.\n"
        "4. Ensure the article follows a logical flow with:\n"
        "   - A compelling introduction that hooks the reader.\n"
        "   - A well-structured body with insightful points and supporting details.\n"
        "   - A strong conclusion that summarizes key takeaways and encourages action.\n"
        "5. Proofread meticulously for grammatical accuracy, readability, and alignment with the brand’s voice and tone."
    ),
    expected_output=(
        "A well-structured and engaging blog post in markdown format, ready for publication. \n"
        "- Each section should contain 2 to 3 well-developed paragraphs.\n"
        "- The content should be clear, concise, and engaging, ensuring consistency with the brand’s style."
    ),
    agent=writer,  # ✅ Assigning the 'writer' agent to execute this task
)


### Task: Edit

In [14]:
edit = Task(
    description=(
        "1. Carefully proofread the given blog post to eliminate grammatical, spelling, and punctuation errors.\n"
        "2. Ensure alignment with the brand’s voice, tone, and style guidelines.\n"
        "3. Verify that the content follows a logical structure and maintains clarity and readability.\n"
        "4. Improve sentence flow, readability, and engagement where necessary without altering the author’s intent.\n"
        "5. Ensure the post adheres to markdown formatting standards for seamless publication.\n"
        "6. Confirm that all sections have well-structured paragraphs (2-3 per section) and engaging headings.\n"
        "7. Cross-check any referenced facts or data for accuracy and consistency."
    ),
    expected_output=(
        "A polished, publication-ready blog post in markdown format, free of grammatical errors.\n"
        "- Well-structured sections with 2 to 3 paragraphs each.\n"
        "- The content should be engaging, clear, and professionally formatted for publication."
    ),
    agent=editor,  # ✅ Assigning the 'editor' agent to execute this task
)


### Step 05. Forming the crew

Crew is **group of AI agents**, collaborating to perform set of tasks.


- `tasks`: list of tasks
- `agents`: list of agents
- `process (optional)`: work flow (e.g., sequential, hierarchical) default : sequential 
- `verbose=2` allows you to see all the logs of the execution. 


    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.



In [25]:
crew = Crew(
    tasks=[plan, write, edit],  # ✅ Ensure these are all Task objects
    agents=[planner, writer, editor],  # ✅ Ensure these are all Agent objects
    verbose=2  # ✅ Enable detailed logging for tracking
)


In [26]:
topic = "Artificial Intelligence"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Research and prioritize the latest trends, key players, and noteworthy news related to Artificial Intelligence.
2. Define the target audience, considering their demographics, interests, and pain points.
3. Develop a structured content outline, including:
   - A compelling introduction.
   - Key discussion points with supporting details.
   - A clear and engaging call to action.
4. Integrate SEO keywords and provide relevant sources for accuracy and credibility.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer:
**Content Plan: Artificial Intelligence**

**I. Introduction**
- Begin with a compelling statistic or anecdote that highlights the transformative impact of AI.
- Define AI and its various applications across industries.
- State the purpose of the article: to explore the latest trends, key players, and challenges in AI.

**II. Key Discussion Points**

**A. Lates

To display the result as markdown in the notebook.

In [27]:
from IPython.display import Markdown
Markdown(result)

# **Artificial Intelligence: Unlocking Limitless Possibilities**

**Introduction**

Artificial Intelligence (AI) is rapidly transforming our world, revolutionizing industries and reshaping human society. From self-driving cars to medical diagnosis, AI applications are proliferating, unlocking endless possibilities. This article delves into the latest trends, key players, and challenges in the AI landscape, providing invaluable insights for tech enthusiasts, business professionals, and anyone seeking to understand the transformative power of AI.

**Latest Trends in AI**

The field of AI is witnessing remarkable advancements, particularly in Machine Learning, Deep Learning, and Natural Language Processing (NLP). These technologies are empowering computers to perform tasks that were once considered exclusively human, such as image and speech recognition, language translation, and decision-making.

Emerging AI technologies, such as Generative AI, Edge AI, and Quantum AI, are further expanding the boundaries of AI capabilities. Generative AI can create realistic images, text, and music, while Edge AI enables real-time AI processing on devices like smartphones. Quantum AI harnesses the power of quantum computing to accelerate AI algorithms and solve complex problems.

**Key Players in the AI Landscape**

Global tech giants like Google, Microsoft, and Amazon are leading the charge in AI research and development. They are investing heavily in AI labs and acquiring AI startups to bolster their capabilities. These companies are driving innovation in various AI domains, including cloud computing, autonomous vehicles, and healthcare.

However, startups and research institutions also play a vital role in AI innovation. They are often at the forefront of cutting-edge research, developing novel AI algorithms and applications. Collaboration between industry leaders and academic institutions fosters a dynamic ecosystem that drives the progress of AI.

**Noteworthy News and Developments**

The field of AI is constantly abuzz with groundbreaking achievements and milestones. Researchers have developed AI systems that surpass human performance in complex games like chess and Go. AI-powered algorithms are also being used to accelerate drug discovery, diagnose diseases, and provide personalized education experiences.

Ethical considerations and societal implications of AI are also gaining attention. As AI systems become more powerful, concerns arise about potential biases, job displacement, and the responsible use of AI. Governments and regulatory bodies are working to establish guidelines and frameworks to ensure the ethical development and deployment of AI.

**Target Audience Analysis**

This article primarily targets individuals with an interest in the impact and advancements of AI. It is designed to appeal to tech enthusiasts, business professionals, policymakers, and anyone seeking to gain a deeper understanding of AI's transformative potential. By addressing their knowledge gaps, concerns, and aspirations, the content aims to provide valuable insights and foster engagement.

**Call to Action**

To stay abreast of the latest developments in AI, consider subscribing to our newsletter or following our social media channels. We encourage you to explore further reading and resources on AI to deepen your knowledge and stay informed about this rapidly evolving field. Your thoughts, comments, and questions are also welcome, as we seek to foster a vibrant and informative discussion about the future of AI.

**Conclusion**

Artificial Intelligence is poised to continue its transformative journey, unlocking unprecedented possibilities and shaping our world. While ethical considerations must be carefully addressed, the potential of AI to improve human lives, enhance efficiency, and drive innovation is undeniable. As we move forward, collaboration and responsible development will be crucial to ensure that AI benefits all of society and empowers us to create a better future.

In [28]:
!pip freeze > requirements.txt

In [30]:
file_path = "ai_topic_result.md"

# Open the file in write mode and save the result in markdown format
with open(file_path, "w") as file:
    # You can optionally format the result as markdown content
    file.write(f"# Topic: {topic}\n\n")
    file.write("## Result:\n\n")
    file.write(str(result))  # Convert the result to string if it's not already

print(f"Result saved to {file_path}")

Result saved to ai_topic_result.md
